In [ ]:
# extract top ranked book id's, etc from url

In [150]:
from sudachipy import tokenizer
from sudachipy import dictionary
import sqlite3
import jaconv
import re
import requests
from requests.exceptions import HTTPError
import json
import time
import traceback
import os

# sqlite example

In [5]:
import sqlite3


con = sqlite3.connect("jmdict.db")

cur = con.cursor()

for row in cur.execute('SELECT furigana FROM jmdict where word = "儀式";'):
    print(row)

con.close()

('0:ぎ;1:しき',)


# jaconv example

In [1]:
import jaconv

# Hiragana to Katakana
jaconv.hira2kata('ともえまみ')
# => 'トモエマミ'

# Hiragana to half-width Katakana
jaconv.hira2hkata('ともえまみ')
# => 'ﾄﾓｴﾏﾐ'

# Katakana to Hiragana
jaconv.kata2hira('巴マミ')

'巴まみ'

# DB to hold Aozora Stats

In [104]:
wordDict = {"kango": ["a", "銀河鉄道"], "wago": ["走れメロス", "b"], "verb": ["走れメロス", "b"],}
wordListJson = json.dumps(wordDict, indent = None, ensure_ascii=False)
print(wordListJson)

{"kango": ["a", "銀河鉄道"], "wago": ["走れメロス", "b"], "verb": ["走れメロス", "b"]}


In [105]:
bookId = 1
title = "test title"
authors = "me, you"
exData = [bookId, title, authors, wordListJson, 4.55, 5]

In [115]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
cur2.execute("DROP TABLE IF EXISTS aozoraStats")

cur2.execute("CREATE TABLE aozoraStats(bookid INTEGER PRIMARY KEY, \
            title TEXT, authors TEXT, wordlist JSON, difficulty FLOAT, length INTEGER)")


In [106]:
cur2.execute("INSERT INTO aozoraStats (bookid, title, authors, wordlist, difficulty) VALUES (?, ?, ?, ?, ?, ?);", exData)
con2.commit()

In [117]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
for row in cur2.execute('SELECT * FROM aozoraStats;'):
    print(row)
    jsonResponse = json.loads(row[3])
    print(jsonResponse["verb"])
con2.close()

(127, '羅生門', '芥川 竜之介', '{"kango": ["日", "下人", "門", "所々", "円柱", "一", "大路", "以上", "市", "烏帽子", "二三", "人", "もの", "何故", "地震", "火事", "饑饉", "災", "洛中", "旧記", "仏像", "仏具", "丹", "金銀", "箔", "木", "料", "始末", "修理", "狐狸", "死人", "習慣", "気味", "近所", "足ぶみ", "何", "羽", "輪", "鴟尾", "胡麻", "勿論", "上", "肉", "刻限", "目", "糞", "点々", "一番", "紺", "頬", "気", "作者", "格別", "主人", "家", "前", "当時", "町", "衰微", "永年", "実", "余波", "途方", "朝", "影響", "明日", "次第", "屋根", "斜", "手段", "築", "土", "死", "何度", "道", "低徊", "逢着", "肯定", "当然", "勇気", "大儀", "遠慮", "晩", "楽", "楼", "眼", "下", "分", "中段", "身", "容子", "火", "蜘蛛", "巣", "天井", "急", "平", "内", "噂", "死骸", "無造作", "範囲", "数", "人間", "事実", "人形", "口", "部分", "一層", "永久", "腐爛", "臭気", "感情", "嗅覚", "背", "老婆", "松", "木片", "顔", "多分", "六", "恐怖", "四", "好奇心", "暫時", "呼吸", "記者", "語", "頭身", "毛", "両手", "丁度", "子", "心", "同時", "憎悪", "語弊", "悪", "反感", "問題", "未練", "善悪", "自分", "一目", "無言", "勝敗", "無理", "眼球", "執拗", "明白", "生死", "全然", "意志", "支配", "意識", "仕事", "円満", "成就", "得意", "満足", "声", "検非違使", "庁", "役人", "旅", "肉食", "物", "存外", "平凡", "失望"

In [3]:
# find top ranked books by year/month

def fetchRankedBooks(date = "2020/01"):
    try:
        response = requests.get(f'https://pubserver2.herokuapp.com/api/v0.1/ranking/txt/{date}')
        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        return jsonResponse

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [8]:
def fetchURL(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        # note this query doesn't contain json
        if response is not None and response.text is not None:
            return response.text
        else:
            return None

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [14]:
%time fetchRankedBooks()

Entire JSON response
Wall time: 4.8 s


In [13]:
rankedBooks = fetchRankedBooks()
for row in rankedBooks:
    authors = ", ".join(row["authors"])
    print("book id " + str(row["book_id"]) + " title " + row["title"] + " authors " + authors)


book id 773 title こころ authors 夏目 漱石
book id 301 title 人間失格 authors 太宰 治
book id 127 title 羅生門 authors 芥川 竜之介
book id 456 title 銀河鉄道の夜 authors 宮沢 賢治
book id 789 title 吾輩は猫である authors 夏目 漱石
book id 2093 title ドグラ・マグラ authors 夢野 久作
book id 57624 title さぶ authors 山本 周五郎
book id 752 title 坊っちゃん authors 夏目 漱石
book id 1465 title 蟹工船 authors 小林 多喜二
book id 92 title 蜘蛛の糸 authors 芥川 竜之介
book id 1565 title 斜陽 authors 太宰 治
book id 57228 title 怪人二十面相 authors 江戸川 乱歩
book id 57817 title 日本婦道記 authors 山本 周五郎
book id 1567 title 走れメロス authors 太宰 治
book id 424 title 檸檬 authors 梶井 基次郎
book id 794 title 三四郎 authors 夏目 漱石
book id 799 title 夢十夜 authors 夏目 漱石
book id 45630 title 〔雨ニモマケズ〕 authors 宮沢 賢治
book id 2078 title 舞姫 authors 森 鴎外
book id 49866 title 変身 authors カフカ フランツ
book id 56648 title 人間椅子 authors 江戸川 乱歩
book id 43737 title 銀河鉄道の夜 authors 宮沢 賢治
book id 776 title 草枕 authors 夏目 漱石
book id 5016 title 源氏物語 authors 紫式部 
book id 52409 title 三国志 authors 吉川 英治
book id 56650 title Ｄ坂の殺人事件 authors 江戸川 乱歩
book

# Sudachi Python Parser Example

In [15]:
# Create the Sudachi Parser.
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C # mode.C is the one that splits things into the smallest form

In [125]:
# this cell is just used to test the parser and help with debugging.

text = "※二十年戦考え二ヶ月べんきょう花火花粉たくさんきれいな寿司が食べたい。勉強をします。音楽がすきですから、よく CD を聞きます。ワンさん"

tokens = tokenizer_obj.tokenize(text, mode)
for token in tokens:
    print("surface {}, dict {}, reading {}, pos {}"\
          .format(token.surface(), token.dictionary_form(), token.reading_form(), token.part_of_speech()))

surface ※, dict ※, reading キゴウ, pos ('補助記号', '一般', '*', '*', '*', '*')
surface 二十, dict 二十, reading ニジュウ, pos ('名詞', '数詞', '*', '*', '*', '*')
surface 年, dict 年, reading ネン, pos ('名詞', '普通名詞', '助数詞可能', '*', '*', '*')
surface 戦, dict 戦, reading セン, pos ('接尾辞', '名詞的', '一般', '*', '*', '*')
surface 考え, dict 考える, reading カンガエ, pos ('動詞', '一般', '*', '*', '下一段-ア行', '連用形-一般')
surface 二, dict 二, reading ニ, pos ('名詞', '数詞', '*', '*', '*', '*')
surface ヶ月, dict ヶ月, reading カゲツ, pos ('名詞', '普通名詞', '助数詞可能', '*', '*', '*')
surface べんきょう, dict べんきょう, reading ベンキョウ, pos ('名詞', '普通名詞', 'サ変可能', '*', '*', '*')
surface 花火, dict 花火, reading ハナビ, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface 花粉, dict 花粉, reading カフン, pos ('名詞', '普通名詞', '一般', '*', '*', '*')
surface たくさん, dict たくさん, reading タクサン, pos ('副詞', '*', '*', '*', '*', '*')
surface きれい, dict きれい, reading キレイ, pos ('形状詞', '一般', '*', '*', '*', '*')
surface な, dict だ, reading ナ, pos ('助動詞', '*', '*', '*', '助動詞-ダ', '連体形-一般')
surface 寿司, dict 寿司, reading

In [96]:
# Removes ruby and some other inserted stuff from the Aozora text that isn't part of the "actual" text itself.
def CleanUPAozoraText(text):
    rubyRegex = """(《.*?》)"""
    text = re.sub(rubyRegex, '', text)
    dashes = "-------------------------------------------------------"
    dashExpression = "(" + dashes + ".*" + dashes + ")"
    text = re.sub(dashExpression, '', text, 0, re.DOTALL)
    indentRegex = """(［＃.*?］)"""
    text = re.sub(indentRegex, '', text,)
    creditsRegex = "(底本：.*)"
    text = re.sub(creditsRegex, '', text, 0, re.DOTALL)
    return text

# Magic to determine Wago vs Kango

In [11]:
def CalcTrueLen(furigana):
    # Function to return the true length of the reading.
    # Certain reading combinations can have multi characters but only count as 1 sound.
    # This likely indicates Kango. This function accounts for those combinations.
    # This link has good info for understanding the logic below. 
    # https://japanese.stackexchange.com/questions/41585/heuristics-for-identifying-japanese-origin-words-%E5%92%8C%E8%AA%9E
    vowels = ["あ", "い", "う", "え", "お",]
    glides1 = ["ょ", "ゅ", "ゃ"]
    glides2 = ["ょう", "ゅう"]
    furiLen = len(furigana)
    if furiLen == 1:
        return 1
    elif furiLen == 2:
        lastChar = furigana[1]
        if "しき" == furigana:
            return 1
        elif lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち":
            return 1
        elif lastChar in vowels:
            return 1
        elif lastChar in glides1:
            return 1
        else:
            return 2
    elif furiLen == 3:
        lastTwo = furigana[1:]
        secondChar = furigana[1]
        lastChar = furigana[2]
        if lastTwo in glides2:
            return 1
        elif not (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            # Second char in the last but last char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        elif (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and not (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            # Last char in the list but second char not. 
            # (should only count as 2 if one or the other char matches the pattern. Not both)
            return 2
        else:
            return 3
    elif furiLen == 4:
        secondChar = furigana[1]
        lastChar = furigana[3]
        if (lastChar == "つ" or lastChar == "く" or lastChar == "ん" or lastChar == "っ" or lastChar == "ち") \
            and (secondChar == "つ" or secondChar == "く" or secondChar == "ん" or secondChar == "っ" or secondChar == "ち"):
            return 2
        else:
            return furiLen
    else:
        #just in case
        return furiLen
    

# Method to extract stats from text

In [168]:
# All of the math below is meant to match with the math used by JReadability
def ExtractStats(tokens, text, includeAuxVerbs = False):
    numVerbs = 0
    numAux = 0
    numAuxVerbs = 0
    numKango = 0
    numWago = 0
    numGairaigo = 0
    sentenceAvgLen = 0
    numPeriods = 0
    numSpaces = 0
    verb = "動詞"
    dependent = "非自立可能"
    aux1 = "助動詞" # counts as an auxVerb
    aux2 = "助詞" # counts as an auxVerb
    auxVerb = "格助詞" # counts as an auxVerb
    adverb = "副詞"
    conjuction = "接続詞"
    symbol = "補助記号"
    space = "空白"
    naAdj = "形状詞"
    iAdj = "形容詞"
    noun = "名詞"
    nounLike = "名詞的"
    properNoun = "固有名詞"
    pronoun = "代名詞"
    counter = "助数詞可能"
    connectingWord = "連体詞"
    suruVerb = "サ変可能"
    suffix = "接尾辞"
    hiraganaUniLow = u'\u3040'
    katakanaUniLow = u'\u30A0' 
    katakanaUniHi = u'\u30FF'
    kangoExceptionList = ["きれい"]
    alphabetRegexp = re.compile(r'[a-zA-Z]')
    numberRegexp = re.compile(r'[0-9]')
    kangoWords = {}
    wagoWords = {}
    verbWords = {}
    gairaigoWords = {}
    totalVerbsList = []
    
    
    for token in tokens:
        #print(token.surface())
        if token.surface() == "。":
            numPeriods += 1
            continue
        
        
        posList = token.part_of_speech()
        if aux2 in posList or auxVerb in posList:
            numAux +=1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif aux1 in posList:
            numAuxVerbs += 1
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
            if includeAuxVerbs:
                # Not sure if auxVerbs should count in the number of total auxillary words.
                # If includeAuxVerbs then add it to the total
                numAux +=1
        elif verb in posList:
            dictForm = token.dictionary_form()
            #JReadability doesn't consider する/いる in the calculation.
            if dependent not in posList:
                numVerbs += 1
                verbWords[dictForm] = 1
                totalVerbsList.append(dictForm)
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif naAdj in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            if reading in kangoExceptionList:
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
            elif reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif adverb in posList:
            surface = token.surface()
            reading = jaconv.kata2hira(token.reading_form())
            #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
            if reading == surface:
                # Hiragana only word, so Wago
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi:
                # considering hiragana/katakana word as Wago.
                numWago += 1
                wagoWords[surface] = 1
                #print("adding {} to numWago".format(surface))
            else:
                # most likely Kango??
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(token.surface()))
        elif conjuction in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif iAdj in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif properNoun in posList:
            # JReadability doesn't count these as wago or kango.
            continue
        elif suffix in posList and not nounLike in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif pronoun in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif counter in posList:
            numWago += 1
            wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif connectingWord in posList:
            continue
            #numWago += 1
            #wagoWords[token.surface()] = 1
            #print("adding {} to numWago".format(token.surface()))
        elif noun in posList or nounLike in posList:
            #complicated case
            surface = token.surface()
            #print("found noun. posList {}".format(posList))
            if alphabetRegexp.search(surface):
                continue
            if numberRegexp.search(surface):
                continue
            elif suruVerb in posList:
                # this checks for nouns that can take the suffix する to become verbs. Those nouns are usually Kango.
                numKango += 1
                kangoWords[surface] = 1
                #print("adding {} to numKango".format(surface))
            elif katakanaUniLow <= min(surface) <= katakanaUniHi:
                # considering katakana nouns as gairaigo
                numGairaigo += 1
                gairaigoWords[surface] = 1
            elif hiraganaUniLow <= min(surface) <= katakanaUniHi and not (hiraganaUniLow <= max(surface) <= katakanaUniHi):
                #word contains hiragana/katakana + kanji. Most likely wago
                numWago += 1
                wagoWords[token.surface()] = 1
            else:
                # Need to look at the reading compared to the surface to determine the character length
                # This is a hueristic so it's certainly not too accurate
                reading = jaconv.kata2hira(token.reading_form())
                #print(f'surface {surface}, reading {reading}, equal? {reading == surface}')
                if reading == surface:
                    # probably a hiragana only word, so wago
                    numWago += 1
                    wagoWords[surface] = 1
                    #print("adding {} to numWago".format(surface))
                    continue
                
                query = 'SELECT furigana FROM jmdict where word = "{}" and reading = "{}";'\
                                  .format(surface, reading)
                #print(query)
                row = cur.execute(query).fetchone()
                if row != None:
                    # Need deal with special cases ん、つ、く、しき
                    try:
                        #print(f"found query result in db {row[0]}")
                        readingLen = 0
                        furiHelper = row[0].split(";") #e.g. '0:ぎ;1:しき'
                        for item in furiHelper:
                            ruby = item.split(":")
                            if "-" not in ruby[0]:
                                #normal case
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                            else:
                                #special case like with 三和土|たたき|0-2:たたき
                                numRange = ruby[0].split("-")
                                # Is there a need to do something special with the numRange??
                                # Doing nothing for now.
                                furigana = ruby[1]
                                readingLen += CalcTrueLen(furigana)
                        
                        #again this is a hueristic and won't always be true
                        #print(f'surface len {len(surface)}, reading len {readingLen}')
                        if len(surface) == readingLen:
                            numKango += 1
                            kangoWords[surface] = 1
                            #print("adding {} to numKango".format(surface))
                        else:
                            numWago += 1
                            wagoWords[surface] = 1
                            #print("adding {} to numWago".format(surface))
                                  
                    except Exception as err:
                        print(f'Error occurred: {err}')
                else:
                    #print(f"couldn't find {surface} {reading}")
                    # word not found in jmdictfurigana so just use the reading
                    if len(surface) == len(reading):
                        numKango += 1
                        kangoWords[surface] = 1
                        #print("adding {} to numKango".format(surface))
                    else:
                        numWago += 1
                        wagoWords[surface] = 1
                        #print("adding {} to numWago".format(surface))
             
        elif space in posList:
            numSpaces += 1
        else:
            # It's probably a symbol. Do nothing
            #print("What dis? {}".format(posList))
            continue
            
    # subtracting the numPeriods from the number of tokens before the divide would make this more accurate
    # but that's not what JReadability does, so leaving it this way for now.
    numTokens = len(tokens) - numSpaces
    # some text in Aozora don't use periods such as poems. Some using single newlines and some use double newlines
    if numPeriods == 0:
        # Check for newlines like in a poem.
        numSingle = text.count(os.linesep)
        numDouble = text.count(os.linesep + os.linesep)
        print(f"numSingle {numSingle}, numDouble {numDouble}, use double {round(numSingle/max(1, numDouble)) == 2}")
        if round(numSingle/max(1, numDouble)) == 2:
            # text likely uses double newlines since the number of single newlines is roughly 2x
            print("using num of double newlines since no periods found")
            numPeriods = numDouble
        else:
            # hard to say, but likely single newlines.
            print("using num of newlines since no periods found")
            numPeriods = numSingle
    
    sentenceAvgLen = numTokens/ max(1, numPeriods)
    uniqueKango = list(kangoWords.keys())
    uniqueWago = list(wagoWords.keys())
    uniqueVerbs = list(verbWords.keys())
    print(f"unique words: kango {len(uniqueKango)}, wago {len(uniqueWago)}, verbs {len(uniqueVerbs)}")
    
    #print(f"{len(kangoWords.keys())} unique kango words found {kangoWords.keys()}")
    #print(f"{len(wagoWords.keys())} unique wago words found {wagoWords.keys()}")
    #print(f"{len(gairaigoWords.keys())} unique gairaigo words found {gairaigoWords.keys()}")
    #print(f"{len(verbWords.keys())} unique verb words found {verbWords.keys()}")
    #print(f"{len(totalVerbsList)} total verb words found {totalVerbsList}")
    print("result avgLen {}, kango {}, wago {}, gairaigo {} verbs {}, auxiliary {}, auxVerbs {}"\
          .format(round(sentenceAvgLen, 2), numKango, numWago, numGairaigo, numVerbs, numAux, numAuxVerbs))
    return (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, \
            uniqueKango, uniqueWago, uniqueVerbs)

# Calc Difficulty

In [13]:
def CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens):
    # Linear Regression Formula from http://jhlee.sakura.ne.jp/papers/lee-et-al2016rb.pdf
    # X = { 平 均 文 長 *-0.056}+{ 漢 語 率 *-0.126}+{ 和 語 率*-0.042}+{動詞率*-0.145}+{助詞率*-0.044}+11.724
    percentKango = numKango / numTokens * 100
    percentWago = numWago / numTokens * 100
    percentVerbs = numVerbs / numTokens * 100
    percentAuxVerbs = numAux / numTokens * 100
    difficulty = sentenceAvgLen*-0.056 + percentKango*-0.126 + percentWago*-0.042 \
        + percentVerbs*-0.145 + percentAuxVerbs*-0.044 + 11.724
    
    return difficulty

# Cell to test the extraction of text stats and compute difficulty (for debug)

In [90]:
# Same logic used in the real extraction for Aozora text, but pulled out into a more easily testable format for debugging.

# connect to jmdictfurigana
con = sqlite3.connect("dbOutput/jmdict.db")
cur = con.cursor()
# call to Aozora goes here...

#text = "音楽がすきですから、よく CD を聞きます。日本がすきですから、日本語を勉強します。安かったですから、買いました。ディズニーランドは楽しかったです。教室は静かでした。わたしはラーメンがすきです。わたしはたばこがきらいです。ワンさんは日本語が上手です。わたしは料理が下手です。"
text = "寿司が食べたい。"
#text = "わたしは料理が下手です。"
#text = "スペインの国境に近いビアリッツにいる姉の一人息子が失明したという通知があったので、大急ぎで知らせなければならないと思ったのである。"

tokens = tokenizer_obj.tokenize(text, mode)
(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) = ExtractStats(tokens, text)

difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
print("difficulty = {}".format(difficulty))

#Store the difficulty and other stats in sqlite

#finally close the connection to jmdictfurigana
con.close()

unique words: kango 1, wago 3, verbs 1
result avgLen 5.0, kango 1, wago 3, gairaigo 0 verbs 1, auxiliary 1, auxVerbs 1
difficulty = 2.6239999999999988


In [189]:
# just for testing.
katakanaUniLow = u'\u30A0' 
katakanaUniHi = u'\u30FF'
word = "ヶ"
if katakanaUniLow <= min(word) <= katakanaUniHi:
    print("katakana")
else:
    print("not katakana")

katakana


In [153]:
# retry with modifications?? # delete this later
CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)

1.6179066974595848

In [54]:
# unicode for kanji = [\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff]

# Grade Aozora Content

In [ ]:
longBooks = []

In [179]:
start = time.time()
try:
    # connect to dbs
    con = sqlite3.connect("dbOutput/jmdict.db")
    cur = con.cursor()
    con2 = sqlite3.connect('dbOutput/aozoraStats.db')
    cur2 = con2.cursor()
    rankedBooks = fetchRankedBooks()
    for row in rankedBooks:
        startTimeThisBook = time.time()
        authors = ", ".join(row["authors"])
        print("\nbook id " + str(row["book_id"]) + " title " + row["title"] + " authors " + authors)

        
        bookId = row["book_id"]
        #check if the book was already scored.
        result = cur2.execute(f'SELECT bookid FROM aozoraStats where bookid = {bookId};').fetchone()
        if result is not None:
            print(f"{bookId} was already added to the db. Skipping it.")
            continue
        if bookId in longBooks:
            print(f"Already found book id {bookId} was too long. Skipping it.")
            continue
            
        # call to Aozora for book text
        url = f"https://pubserver2.herokuapp.com/api/v0.1/books/{bookId}/content?format=txt"
        text = fetchURL(url)
        if text is None:
            print(f"failed to get data for book id {bookId}")
        else:
            text = CleanUPAozoraText(text)
            bookLen = len(text.encode('utf-8'))
            sudachiTokenLimit = 49149
            if bookLen > sudachiTokenLimit:
                print(f"Book {bookId} is length {bookLen} which exceeds the SudachiPy Tokenizer limit {sudachiTokenLimit}." +\
                      "Skipping it.")
                longBooks.append(bookId)
                time.sleep(10)
                continue
            else:
                tokens = tokenizer_obj.tokenize(text, mode)
                (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) \
                    = ExtractStats(tokens, text)

                difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
                print("text len {}, difficulty = {}".format(bookLen, round(difficulty, 2)))

                # Create json unique words obj
                uniqueWordDict = {"kango": uniqueKango, "wago": uniqueWago, "verb": uniqueVerbs}
                wordListJson = json.dumps(uniqueWordDict, indent = None, ensure_ascii=False)
                bookStats = [row["book_id"], row["title"], authors, wordListJson, round(difficulty, 2), numTokens]
                
                #Store the difficulty and other stats in sqlite
                cur2.execute("INSERT INTO aozoraStats (bookid, title, authors, wordlist, difficulty, length) VALUES (?, ?, ?, ?, ?, ?);",\
                             bookStats)
                con2.commit()
                endTimeThisBook = time.time()
                print(f"{endTimeThisBook - startTimeThisBook} seconds to process book {bookId}.")
                print(f"Elapsed time is {round((endTimeThisBook - start)/60, 2)} minutes")
                # the Aozora API site is throwing 404 after some time maybe thinking I'm trying to DOS them.
                # adding a wait to space out calls to fetch by ~10 seconds
                time.sleep(10- (endTimeThisBook - startTimeThisBook))
                
                
    print(f"{len(longBooks)} long books were skipped.")

    
except Exception as err:
    print(f'Error occurred: {err}')
    print(traceback.format_exc())
finally:
    #finally close the connection to jmdictfurigana
    con.close()
    con2.close()
    end = time.time()
    print(f"total time {round((end - start)/60, 2)} minutes")



book id 773 title こころ authors 夏目 漱石
Already found book id 773 was too long. Skipping it.

book id 301 title 人間失格 authors 太宰 治
Already found book id 301 was too long. Skipping it.

book id 127 title 羅生門 authors 芥川 竜之介
127 was already added to the db. Skipping it.

book id 456 title 銀河鉄道の夜 authors 宮沢 賢治
Already found book id 456 was too long. Skipping it.

book id 789 title 吾輩は猫である authors 夏目 漱石
Already found book id 789 was too long. Skipping it.

book id 2093 title ドグラ・マグラ authors 夢野 久作
Already found book id 2093 was too long. Skipping it.

book id 57624 title さぶ authors 山本 周五郎
Already found book id 57624 was too long. Skipping it.

book id 752 title 坊っちゃん authors 夏目 漱石
Already found book id 752 was too long. Skipping it.

book id 1465 title 蟹工船 authors 小林 多喜二
Already found book id 1465 was too long. Skipping it.

book id 92 title 蜘蛛の糸 authors 芥川 竜之介
92 was already added to the db. Skipping it.

book id 1565 title 斜陽 authors 太宰 治
Already found book id 1565 was too long. Skipping it.



numSingle 76, numDouble 13, use double False
using num of newlines since no periods found
unique words: kango 20, wago 163, verbs 38
result avgLen 6.12, kango 24, wago 431, gairaigo 0 verbs 49, auxiliary 142, auxVerbs 64
text len 2537, difficulty = 3.97
1.0507948398590088 seconds to process book 59056.
Elapsed time is 0.27 minutes

book id 56647 title 二銭銅貨 authors 江戸川 乱歩
Already found book id 56647 was too long. Skipping it.

book id 52415 title 三国志 authors 吉川 英治
Already found book id 52415 was too long. Skipping it.

book id 59641 title 薄口醤油 authors 北大路 魯山人
59641 was already added to the db. Skipping it.

book id 52417 title 三国志 authors 吉川 英治
Already found book id 52417 was too long. Skipping it.

book id 52419 title 三国志 authors 吉川 英治
Already found book id 52419 was too long. Skipping it.

book id 882 title アッシャー家の崩壊 authors ポー エドガー・アラン
Already found book id 882 was too long. Skipping it.

book id 59429 title 茶碗の中 authors 小泉 八雲
59429 was already added to the db. Skipping it.

book id 

numSingle 840, numDouble 413, use double True
using num of double newlines since no periods found
unique words: kango 245, wago 1403, verbs 344
result avgLen 17.42, kango 664, wago 6094, gairaigo 2 verbs 706, auxiliary 2097, auxVerbs 539
text len 33900, difficulty = 3.32
1.945202350616455 seconds to process book 51307.
Elapsed time is 0.95 minutes

book id 52396 title 宮本武蔵 authors 吉川 英治
Already found book id 52396 was too long. Skipping it.

book id 4872 title 愛読書の印象 authors 芥川 竜之介
4872 was already added to the db. Skipping it.

book id 2381 title 瓶詰地獄 authors 夢野 久作
2381 was already added to the db. Skipping it.

book id 52182 title 断腸亭日乗 authors 永井 荷風
52182 was already added to the db. Skipping it.

book id 52181 title 断腸亭日乗 authors 永井 荷風
numSingle 12, numDouble 4, use double False
using num of newlines since no periods found
unique words: kango 25, wago 47, verbs 4
result avgLen 9.83, kango 28, wago 74, gairaigo 0 verbs 4, auxiliary 31, auxVerbs 3
text len 552, difficulty = 3.9
1.209

numSingle 1, numDouble 0, use double False
using num of newlines since no periods found
unique words: kango 40, wago 44, verbs 1
result avgLen 243.0, kango 46, wago 45, gairaigo 0 verbs 1, auxiliary 5, auxVerbs 0
text len 1808, difficulty = -5.2
1.37770414352417 seconds to process book 46187.
Elapsed time is 1.61 minutes

book id 52398 title 宮本武蔵 authors 吉川 英治
Already found book id 52398 was too long. Skipping it.

book id 47876 title ふるさとびと authors 堀 辰雄
47876 was already added to the db. Skipping it.

book id 48815 title 或新年の小説評 authors 田山 花袋, 田山 録弥
48815 was already added to the db. Skipping it.

book id 57343 title 鏡地獄 authors 江戸川 乱歩
57343 was already added to the db. Skipping it.

book id 58337 title いやな感じ authors 高見 順
numSingle 1, numDouble 0, use double False
using num of newlines since no periods found
unique words: kango 78, wago 91, verbs 2
result avgLen 1182.0, kango 171, wago 137, gairaigo 0 verbs 2, auxiliary 21, auxVerbs 4
text len 3108, difficulty = -56.88
2.0704188346862

unique words: kango 20, wago 48, verbs 6
result avgLen 160.0, kango 26, wago 110, gairaigo 5 verbs 8, auxiliary 59, auxVerbs 14
text len 776, difficulty = -4.52
0.9957702159881592 seconds to process book 55926.
Elapsed time is 2.11 minutes

book id 389 title たけくらべ authors 樋口 一葉
Already found book id 389 was too long. Skipping it.

book id 2256 title 姥捨 authors 太宰 治
2256 was already added to the db. Skipping it.

book id 5021 title 源氏物語 authors 紫式部 
Already found book id 5021 was too long. Skipping it.

book id 1726 title 大菩薩峠 authors 中里 介山
Already found book id 1726 was too long. Skipping it.

book id 42338 title 盗まれた手紙 authors ポー エドガー・アラン
Already found book id 42338 was too long. Skipping it.

book id 57542 title 赤ひげ診療譚 authors 山本 周五郎
Already found book id 57542 was too long. Skipping it.

book id 58278 title 水底の感 authors 夏目 漱石
58278 was already added to the db. Skipping it.

book id 58699 title 悪霊物語 authors 江戸川 乱歩
58699 was already added to the db. Skipping it.

book id 1571 title きり


book id 4376 title 即興詩人 authors アンデルセン ハンス・クリスチャン
Book 4376 is length 784886 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 46845 title 人生論ノート authors 三木 清
Book 46845 is length 214225 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 120 title 女 authors 芥川 竜之介
unique words: kango 87, wago 291, verbs 92
result avgLen 27.92, kango 178, wago 969, gairaigo 1 verbs 102, auxiliary 385, auxVerbs 131
text len 5911, difficulty = 3.08
1.1631319522857666 seconds to process book 120.
Elapsed time is 7.26 minutes

book id 1076 title 倫敦塔 authors 夏目 漱石
unique words: kango 644, wago 1602, verbs 491
result avgLen 22.79, kango 1311, wago 7661, gairaigo 60 verbs 941, auxiliary 3221, auxVerbs 824
text len 47062, difficulty = 3.12
1.924736738204956 seconds to process book 1076.
Elapsed time is 7.44 minutes

book id 2380 title 死後の恋 authors 夢野 久作
Book 2380 is length 55079 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 47296 title ランボオ詩集


book id 56671 title 青銅の魔人 authors 江戸川 乱歩
Book 56671 is length 248072 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 791 title 野分 authors 夏目 漱石
Book 791 is length 283837 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 1543 title 般若心経講義 authors 高神 覚昇
Book 1543 is length 319165 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 1738 title 弟子 authors 中島 敦
Book 1738 is length 72238 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 42604 title レ・ミゼラブル authors ユゴー ヴィクトル
Book 42604 is length 858699 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 57320 title アリスはふしぎの国で authors キャロル ルイス
Book 57320 is length 231067 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 226 title ボヘミアの醜聞 authors ドイル アーサー・コナン
Book 226 is length 56378 which exceeds the SudachiPy Tokenizer limit 49149.Skipping it.

book id 5025 title 源氏物語 authors 紫式部 
Book 5025 is length 87674 which ex

In [180]:
# get count of books in the db
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
for row in cur2.execute('SELECT count(*) FROM aozoraStats;'):
    print(row)
con2.close()

(218,)


In [186]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
for row in cur2.execute('SELECT difficulty, bookid FROM aozoraStats order by difficulty desc;'):
    print(row)
con2.close()

(6.12, 45630)
(5.66, 58278)
(5.07, 42311)
(4.88, 43752)
(4.87, 1927)
(4.87, 52976)
(4.87, 58875)
(4.86, 258)
(4.86, 42315)
(4.85, 43754)
(4.84, 46605)
(4.79, 236)
(4.69, 2256)
(4.68, 466)
(4.68, 58882)
(4.68, 59636)
(4.66, 59742)
(4.64, 240)
(4.64, 473)
(4.62, 46597)
(4.6, 1572)
(4.58, 1578)
(4.56, 59819)
(4.52, 464)
(4.52, 628)
(4.49, 1562)
(4.48, 51344)
(4.46, 52973)
(4.44, 1567)
(4.43, 59480)
(4.4, 42308)
(4.35, 58277)
(4.33, 42378)
(4.32, 14)
(4.32, 35)
(4.31, 277)
(4.31, 470)
(4.29, 15)
(4.29, 270)
(4.27, 45381)
(4.22, 129)
(4.22, 42314)
(4.22, 42376)
(4.2, 59055)
(4.18, 57197)
(4.17, 832)
(4.17, 55302)
(4.15, 54836)
(4.15, 58699)
(4.14, 51005)
(4.12, 1607)
(4.11, 179)
(4.08, 1571)
(4.08, 2317)
(4.07, 2381)
(4.05, 52182)
(4.0, 50326)
(3.99, 260)
(3.97, 55)
(3.97, 59056)
(3.96, 21)
(3.96, 43014)
(3.95, 57192)
(3.94, 56)
(3.93, 2569)
(3.93, 60082)
(3.92, 16)
(3.92, 242)
(3.9, 52181)
(3.9, 55915)
(3.86, 59231)
(3.83, 59058)
(3.82, 59384)
(3.81, 241)
(3.8, 411)
(3.79, 43016)
(3.79, 51

In [182]:
#bookId = 51307 # negative score b/c no periods. uses double newlines.
#bookId = 45630 # negative score b/c no preiods. Uses single newlines.
#bookId = 58278 # just a random book
bookId = 55112 # this has an image in the data, so that kinda makes the text response really messed up...
url = f"https://pubserver2.herokuapp.com/api/v0.1/books/{bookId}/content?format=txt"
text = fetchURL(url)
text = CleanUPAozoraText(text)

In [183]:
print(text)

臼NG

IHDR   ﾇ  �   �ｵ-   (tEXtCreation Time 木 7 6 2018 16:34:12 +0900J^渦   tIME�#ﾊ�	�   	pHYs    ﾒﾝ~�   gAMA  ｱ��a   PLTE   ���･ﾙ渤  EIDATxﾚ�ﾚﾁｯ紮pｿu�ｷRY�黒xﾍｬ�nT箚�4Sﾔk･�Z�謹�ｷj塰*H菰C�ﾏ�p,R鰺SｱZ�ﾆ･j勅ﾔｱk煙ﾉｼ��岾度ﾆ�R*т �ｼ��yﾟｱcﾏ�翔℃�t�G%�毘ﾈｸ�)oﾙ-'�9kqﾒ�ﾎｿWX�p�>UｶｭB�*IｧD-�:>>/:藐昶wﾊ7ｻD~ｿMn{�ﾒ�69Gm{ﾍﾏ躱ﾂ売ｭみｪh鏑uｸｬM
ﾕoｪﾞﾘ"sﾚﾚZ0｣ｭ�Aｧ|ﾘﾞﾊｲ#ﾃ烽uﾉ�ｵｷ�ｭ"�"ｲﾟ&ｩﾊ;2ｴ[茆暈�_ﾋp%EﾇｾQaIﾙﾋS&;ｻ�%ﾓR�\�:%oN`jK�氤ﾇﾝﾂ腸ﾇiｩ�Bﾙｶ	8執ｳ2ｴﾜｷ1JK�禀t�ﾂ搏ﾄY嬉ｮ綻ﾒ&p2I�壻Gｻqｾ1ﾅ/ｷ5�穃-ョ]ｾｪsﾗ)ﾉ�"�ﾇsZЯ�6ｪ	�薐ｰ颯ﾉｶﾜﾝ屹$･ｨ�*a浜Jｹ.J奄c諂窈zVAﾙE鶺-ｫ�D2(｢ｰｶo1�g瀬ﾈ�O%｡:UQ%躬ｮCu�
Bﾋn｡蘓]ｵ�冪/�ｼ�'OzRI礁ｪ�ﾘBｴP-CｨIH@]ｬｴ｡�老Ｐa鯉�R�7$堋�*13季ｰ*[ﾃﾌｵﾕy)%�)黄ｼｨK<cｫrﾎﾕ�;ｸ様F描γ��齟魔ﾗ� ;lM堺Rｒ縉'監� GY9$ﾅxﾝ�� r盤Q|�辟h"rY典K6-･榱Z��.$(�G英<%�ﾁT� 7<ﾜ]ｸｸP~ｴ鮗j?]�娜ﾉ'�1躁p��ﾉP�ﾔ⊥A�ﾎAiQﾗEOｰ弧�d<ﾟg<ﾉR*v兢)况
ﾌz?�T�
ﾃ*ﾏﾁ�､ﾓE)ﾟ沈ｵ揀|j��ｭX�ｨﾖﾌ�ｩｷj&･�3ﾀ!k灰yBGX�壺ｮ %ﾒ包ｬ&E)a#K8
�ﾃp)�ﾘﾚｬｶｼｯт�ﾄﾊﾋｯ玖�呉ﾁ�8`�y
%羲ﾜRB.%AMd��ﾎ審悁ﾇ蟯�ﾛ)ﾔ�吶ｱｾ縺T"ｿt髀ｵﾁNI-#ｽｱ[��篁垪3ﾇﾇﾂ?慰m�ｴ&p�:ﾎP吸魘�9瘤ﾓCY�ｳｺ�#a�\<αｨﾇA _pﾔ韓iﾈ_a�融ﾎ�ｭﾅ

In [142]:
fetchURL(url)

'〔雨ニモマケズ〕\r\n宮澤賢治\r\n\r\n-------------------------------------------------------\r\n【テキスト中に現れる記号について】\r\n\r\n［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定\r\n\u3000\u3000\u3000（数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）\r\n（例）※［＃「「蔭」の「陰のつくり」に代えて「人がしら／髟のへん」、第4水準2-86-78］\r\n-------------------------------------------------------\r\n\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダヲモチ\r\n慾ハナク\r\n決シテ瞋ラズ\r\nイツモシヅカニワラッテヰル\r\n一日ニ玄米四合ト\r\n味噌ト少シノ野菜ヲタベ\r\nアラユルコトヲ\r\nジブンヲカンジョウニ入レズニ\r\nヨクミキキシワカリ\r\nソシテワスレズ\r\n野原ノ松ノ林ノ※［＃「「蔭」の「陰のつくり」に代えて「人がしら／髟のへん」、第4水準2-86-78］ノ\r\n小サナ萓ブキノ小屋ニヰテ\r\n東ニ病気ノコドモアレバ\r\n行ッテ看病シテヤリ\r\n西ニツカレタ母アレバ\r\n行ッテソノ稲ノ朿ヲ［＃「朿ヲ」はママ］負ヒ\r\n南ニ死ニサウナ人アレバ\r\n行ッテコハガラナクテモイヽトイヒ\r\n北ニケンクヮヤソショウガアレバ\r\nツマラナイカラヤメロトイヒ\r\nヒドリノトキハナミダヲナガシ\r\nサムサノナツハオロオロアルキ\r\nミンナニデクノボートヨバレ\r\nホメラレモセズ\r\nクニモサレズ\r\nサウイフモノニ\r\nワタシハナリタイ\r\n\r\n南無無辺行菩薩\r\n南無上行菩薩\r\n南無多宝如来\r\n南無妙法蓮華経\r\n南無釈迦牟尼仏\r\n南無浄行菩薩\r\n南無安立行菩薩\r\n\r\n\r\n\r\n底本：「【新】校本宮澤賢治全集\u3000第十三巻（上）覚書・手帳\u3000本文篇」筑摩書房\r\n\u3000\u3000\u30001997（平成9）年7月30日初版第1刷発行\r\n※本文については写真版を含む本書によった。また、

In [169]:
try:
    # connect to dbs
    con = sqlite3.connect("dbOutput/jmdict.db")
    cur = con.cursor()
    con2 = sqlite3.connect('dbOutput/aozoraStats.db')
    cur2 = con2.cursor()
    tokens = tokenizer_obj.tokenize(text, mode)
    (sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens, uniqueKango, uniqueWago, uniqueVerbs) \
        = ExtractStats(tokens, text)

    difficulty = CalcDifficulty(sentenceAvgLen, numKango, numWago, numVerbs, numAux, numTokens)
    print("text len {}, difficulty = {}".format(bookLen, round(difficulty, 2)))

except Exception as err:
    print(f'Error occurred: {err}')
    print(traceback.format_exc())
finally:
    #finally close the connection to jmdictfurigana
    con.close()
    con2.close()

numSingle 46, numDouble 5, use double False
using num of newlines since no periods found
unique words: kango 25, wago 26, verbs 0
result avgLen 2.59, kango 34, wago 39, gairaigo 35 verbs 0, auxiliary 13, auxVerbs 0
text len 19102, difficulty = 6.12


In [185]:
longBooks

[773,
 301,
 456,
 789,
 2093,
 57624,
 752,
 1465,
 1565,
 57228,
 794,
 49866,
 43737,
 776,
 56650,
 59300,
 2253,
 52410,
 275,
 47061,
 58093,
 935,
 42286,
 52504,
 51215,
 56866,
 60202,
 69,
 56669,
 56645,
 56646,
 42618,
 4803,
 50909,
 56651,
 816,
 60,
 522,
 52411,
 56143,
 46669,
 1504,
 52016,
 56698,
 52412,
 761,
 52413,
 1502,
 51732,
 1869,
 52414,
 605,
 56647,
 52415,
 52417,
 52419,
 882,
 52416,
 52418,
 42934,
 52420,
 57405,
 4673,
 42621,
 1058,
 57649,
 50131,
 695,
 785,
 1317,
 57612,
 52421,
 57181,
 307,
 42773,
 57105,
 56041,
 647,
 946,
 57788,
 56642,
 1746,
 2282,
 1669,
 57347,
 760,
 4328,
 5017,
 772,
 56461,
 521,
 1737,
 56649,
 201,
 775,
 59519,
 782,
 52396,
 783,
 46322,
 689,
 1566,
 42377,
 56873,
 42600,
 56568,
 255,
 1924,
 765,
 42941,
 57841,
 496,
 859,
 219,
 56146,
 204,
 51731,
 673,
 49862,
 57583,
 59900,
 43522,
 58463,
 59071,
 1154,
 48222,
 5019,
 49296,
 56673,
 57240,
 57567,
 57515,
 1505,
 774,
 5020,
 4618,
 8,
 47057,


In [172]:
con2 = sqlite3.connect('dbOutput/aozoraStats.db')
cur2 = con2.cursor()
sql = 'DELETE FROM aozoraStats where difficulty < 0.5'
cur2 = con2.cursor()
cur2.execute(sql)
con2.commit()
con2.close()